In [3]:
from base import SPDataFrame
from pyspark.sql import SparkSession
import os
import geopandas as gpd

ss_builder = SparkSession.builder.appName('t')

ss_builder.config('spark.driver.extraClassPath',
                  '/Users/taypark/Repositories/spark-plugin/resource'
                  '/mysql-connector-java-8.0.26/mysql-connector-java-8.0.26.jar')\
    .config('spark.driver.memory', '14g')\
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true')\
    .config('spark.driver.maxResultSize', 0)

ss = ss_builder.getOrCreate()

# print(ss.sparkContext.getConf().getAll())

gyeonggi_table_df = SPDataFrame.get_db_df_by_tablenames(ss, ['integrated_address_gyeonggi'],
                                          driver='com.mysql.cj.jdbc.Driver',
                                          url='jdbc:mysql://localhost:3306/sparkplus',
                                          user='root',
                                          password='sparkplus')


21/10/12 05:16:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[('spark.app.name', 't'), ('spark.app.startTime', '1633983376848'), ('spark.executor.id', 'driver'), ('spark.driver.host', 'localhost'), ('spark.sql.warehouse.dir', 'file:/Users/taypark/Repositories/spark-plugin/sparkplus/core/spark-warehouse'), ('spark.driver.extraClassPath', '/Users/taypark/Repositories/spark-plugin/resource/mysql-connector-java-8.0.26/mysql-connector-java-8.0.26.jar'), ('spark.driver.port', '54511'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.maxResultSize', '0'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.driver.memory', '14g'), ('spark.ui.showConsoleProgress', 'true'), ('spark.sql.execution.arrow.pyspark.enabled', 'true'), ('spark.app.id', 'local-1633983377979')]


In [4]:
gyeonggi_table_df.printSchema()

root
 |-- manage_number: string (nullable = true)
 |-- roadname_code: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- sido: string (nullable = true)
 |-- sigungu: string (nullable = true)
 |-- eupmyeondong: string (nullable = true)
 |-- bupjungli: string (nullable = true)
 |-- roadname: string (nullable = true)
 |-- is_basement: string (nullable = true)
 |-- building_primary_number: integer (nullable = true)
 |-- building_secondary_number: integer (nullable = true)
 |-- bupjungdong_code: string (nullable = true)



In [4]:
# SPDataFrame(gyeonggi_table_df).address_to_h3(addr_col_name='is_basement').show()
import geopandas as gpd
from pyspark.sql.functions import lit

# gyeonggi_table_df.limit(1)\
#     .withColumn('point', lit(gpd.points_from_xy(37.3211047, 126.9889655))).show()

PREFIX = (37.32, 126.98)

from faker import Faker

fake = Faker()

xy_box = []
for i in range(50):
    x, y = PREFIX
    px, py = str(x), str(y)
    px, py = px + str(fake.random_int(0, 99999)), py + str(fake.random_int(0, 99999))
    xy_box.append((px, py))

    
    

In [5]:
coord_df = ss.createDataFrame(xy_box)
coord_df = coord_df.withColumnRenamed('_1', 'lat').withColumnRenamed('_2', 'lng')

import os 
PARQUET_PATH = os.getcwd() + '/../../resource/Gyeonggi.parquet'

gyeonggi_gdf = gpd.read_parquet(PARQUET_PATH)

print(gyeonggi_gdf)

                         PNU     JIBUN BCHK  SGG_OID COL_ADM_SE  \
0        4111710300101670003    167-3전    1   270697      41110   
1        4111113800101980001    198-1전    1   270698      41110   
2        4111710300201190028  산119-28임    1   270699      41110   
3        4111710300109070001    907-1잡    1   698166      41110   
4        4111710300101770017   177-17임    1   270701      41110   
...                      ...       ...  ...      ...        ...   
5078153  4183040033101950006   195-6 전    1  1892811      41830   
5078154  4183039521200440024  산44-24 임    1  1893383      41830   
5078155  4183036023102440000      244전    1  1885066      41830   
5078156  4183036023102440004   244-4 전    1  1885067      41830   
5078157  4183036023102440003   244-3 전    1  1885068      41830   

                                                  geometry  
0        POLYGON ((127.05529 37.28866, 127.05533 37.288...  
1        POLYGON ((127.01543 37.32614, 127.01547 37.326...  
2        POL

In [2]:
# 개발용

pandas_coord_to_df = coord_df.toPandas()

# print(pandas_coord_to_df)
print(pandas_coord_to_df)

# point_sdf_to_geodataframe = gpd.GeoDataFrame(coord_df, geometry=gpd.points_from_xy(pandas_coord_to_df.lat, pandas_coord_to_df.lng))

# print(point_sdf_to_geodataframe)

temp_list = []

# for i in point_sdf_to_geodataframe.index:
#     for j in gyeonggi_gdf.index:
#         if gyeonggi_gdf.geometry[j].contains(point_sdf_to_geodataframe[i]):
#             temp_list.append(gyeonggi_gdf.EMD_CD)

print(temp_list)

NameError: name 'coord_df' is not defined